In [35]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# 1. Cargar datos
data = pd.read_csv(r'Preprocesamiento\sample1.csv')  # usa r'' para evitar problemas con '\'

# 2. Matriz de características (4000, 9)
Xdata = data[['r1','phi1','z1','r2','phi2','z2','r3','phi3','z3']].values

# 3. Labels
Ydata = data['label'].values.astype(int)

# 4. Estandarizar las 9 características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(Xdata)

# 5. PCA de 9D -> 3D
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

print("Shape original:", Xdata.shape)    # (n_muestras, 9)
print("Shape reducida:", X_pca.shape)    # (n_muestras, 3)

print("Varianza explicada por componente:", pca.explained_variance_ratio_)
print("Varianza explicada total:", pca.explained_variance_ratio_.sum())


Shape original: (4000, 9)
Shape reducida: (4000, 3)
Varianza explicada por componente: [0.33508414 0.31586445 0.2317071 ]
Varianza explicada total: 0.8826556878167358


In [36]:
data_red= pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'])
data_red['label'] = Ydata
data_red.head()

,PC1,PC2,PC3,label
0,-1.495654,1.055660,0.098388,0
1,-1.048163,0.041889,-0.103060,1
2,-1.532818,1.614287,1.980828,0
3,-1.691349,1.531073,0.213513,1
4,0.820857,2.635866,0.454741,1


In [5]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer
import matplotlib.pyplot as plt

#Semilla
np.random.seed(42)

# Función para calcular la matriz de densidad
def density_matrix(state):
    """Calcula la matriz de densidad a partir de un vector de estado  

    Args:
        state (array[complex]): arreglo que representa el vector de estado
    Returns:
        dm: (array[complex]): arreglo que representa la matriz de densidad
    """
    return state * np.conj(state).T


label_0 = [[1], [0]]
label_1 = [[0], [1]]
state_labels = np.array([label_0, label_1], requires_grad=False)

Circuito del QNN con data reloading y fidelity

In [6]:
dev = qml.device("lightning.qubit", wires=1)


@qml.qnode(dev, diff_method="adjoint")
def qcircuit(params, x, y):
    """Circuito variacional con data reloading.

    Args:
        params (array[float]): arreglo de parámetros
        x (array[float]): input
        y (array[float]): etiquetas

    Returns:
        float: fidelity entre la salida del circuito y la etiqueta
    """
    for p in params:
        qml.Rot(*x, wires=0)
        qml.Rot(*p, wires=0)
    return qml.expval(qml.Hermitian(y, wires=[0]))


def cost(params, x, y, state_labels=None):
    """Función de costo

    Args:
        params (array[float]): arreglo de parámetros
        x (array[float]): 2-d arreglo de inputs
        y (array[float]): 1-d arreglo de etiquetas
        state_labels (array[float]): arreglo de etiquetas como vectores de estado

    Returns:
        float: costo promedio sobre el batch
    """
    # Compute prediction for each input in data batch
    loss = 0.0
    dm_labels = [density_matrix(s) for s in state_labels]
    for i in range(len(x)):
        f = qcircuit(params, x[i], dm_labels[y[i]])
        loss = loss + (1 - f) ** 2
    return loss / len(x)

In [7]:
def test(params, x, y, state_labels=None):
    """
    Testeo

    Args:
        params (array[float]): arreglo de parametros
        x (array[float]): 2-d arreglo de input vectors
        y (array[float]): 1-d arreglo de targets
        state_labels (array[float]): 1-d arreglo de representaciones de estado para las etiquetas
    Returns:
        predicted (array([int]): labels predichas para los datos de test
        output_states (array[float]): estados cuanticos de salida del circuito
    """
    fidelity_values = []
    dm_labels = [density_matrix(s) for s in state_labels]
    predicted = []

    for i in range(len(x)):
        fidel_function = lambda y: qcircuit(params, x[i], y)
        fidelities = [fidel_function(dm) for dm in dm_labels]
        best_fidel = np.argmax(fidelities)

        predicted.append(best_fidel)
        fidelity_values.append(fidelities)

    return np.array(predicted), np.array(fidelity_values)


def accuracy_score(y_true, y_pred):
    """Precisión

    Args:
        y_true (array[float]): 1-d arreglo de targets
        y_predicted (array[float]): 1-d arreglo de predicciones
        state_labels (array[float]): 1-d arreglo de representaciones de estado para las etiquetas

    Returns:
        score (float): fraccion de muestras clasificadas correctamente
    """
    score = y_true == y_pred
    return score.sum() / len(y_true)


def iterate_minibatches(inputs, targets, batch_size):
    """
    generador de batches

    Args:
        inputs (array[float]): data de entrada
        targets (array[float]): targets

    Returns:
        inputs (array[float]): un batch de inputs de longitud `batch_size`
        targets (array[float]): un batch de targets de longitud `batch_size`
    """
    for start_idx in range(0, inputs.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield inputs[idxs], targets[idxs]

In [37]:
#Datos de entrenamiento y test
import pandas as pd
from sklearn.model_selection import train_test_split

Xdata=data_red[['PC1','PC2','PC3']].values
Ydata=(data_red['label'].values).astype(int)
#Division para 1 ejemplo:
X_train, X_test, y_train, y_test = train_test_split(Xdata, Ydata, test_size=0.2, random_state=42)
X_train = np.array(X_train, requires_grad=False)
X_test = np.array(X_test, requires_grad=False)
y_train = np.array(y_train, requires_grad=False)
y_test = np.array(y_test, requires_grad=False)

In [13]:
# Entrenamiento del circuito clasificador
num_layers = 6
learning_rate = 0.01
epochs = 10
batch_size = 32

opt = AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999)


# Inicializacion con parametros aleatorios
params = np.random.uniform(size=(num_layers, 3), requires_grad=True)
predicted_train, fidel_train = test(params, X_train, y_train, state_labels)
accuracy_train = accuracy_score(y_train, predicted_train)

predicted_test, fidel_test = test(params, X_test, y_test, state_labels)
accuracy_test = accuracy_score(y_test, predicted_test)

# guardamos las predicciones con pesos aleatorios para comparación
initial_predictions = predicted_test

loss = cost(params, X_test, y_test, state_labels)

print(
    "Epoch: {:2d} | Cost: {:3f} | Train accuracy: {:3f} | Test Accuracy: {:3f}".format(
        0, loss, accuracy_train, accuracy_test
    )
)


Epoch:  0 | Cost: 0.326705 | Train accuracy: 0.510312 | Test Accuracy: 0.507500


In [14]:
import time
max_accuracy=0
inicio_0=time.time()

for it in range(epochs):
    inicio=time.time()
    for Xbatch, ybatch in iterate_minibatches(X_train, y_train, batch_size=batch_size):
        params, _, _, _ = opt.step(cost, params, Xbatch, ybatch, state_labels)

    predicted_train, fidel_train = test(params, X_train, y_train, state_labels)
    accuracy_train = accuracy_score(y_train, predicted_train)
    loss = cost(params, X_train, y_train, state_labels)

    predicted_test, fidel_test = test(params, X_test, y_test, state_labels)
    accuracy_test = accuracy_score(y_test, predicted_test)
    res = [it + 1, loss, accuracy_train, accuracy_test]
    print(
        "Epoch: {:2d} | Loss: {:3f} | Train accuracy: {:3f} | Test accuracy: {:3f}".format(
            *res
        )
    )
    if accuracy_test>max_accuracy:
        max_accuracy=accuracy_test
        max_params=params
    fin=time.time()
    print("Tiempo de epoch:",fin-inicio,"segundos")
    print("-------------------------------------------------------------------")

fin_0=time.time()
print("Tiempo total de entrenamiento:",(fin_0-inicio_0)/60," minutos")
print("Mejor accuracy de test:",max_accuracy)
print("Mejores parametros:",max_params)
params_opt=max_params

Epoch:  1 | Loss: 0.264522 | Train accuracy: 0.610625 | Test accuracy: 0.610000
Tiempo de epoch: 34.54183316230774 segundos
-------------------------------------------------------------------
Epoch:  2 | Loss: 0.246032 | Train accuracy: 0.635938 | Test accuracy: 0.621250
Tiempo de epoch: 34.87286972999573 segundos
-------------------------------------------------------------------
Epoch:  3 | Loss: 0.231864 | Train accuracy: 0.659062 | Test accuracy: 0.636250
Tiempo de epoch: 33.17360830307007 segundos
-------------------------------------------------------------------
Epoch:  4 | Loss: 0.226326 | Train accuracy: 0.665312 | Test accuracy: 0.653750
Tiempo de epoch: 32.928606033325195 segundos
-------------------------------------------------------------------
Epoch:  5 | Loss: 0.223490 | Train accuracy: 0.674687 | Test accuracy: 0.647500
Tiempo de epoch: 33.39457583427429 segundos
-------------------------------------------------------------------
Epoch:  6 | Loss: 0.221185 | Train accu

### II) 1-n EQK 

Quiero ejecutar esto, donde calculo un kernel cuantico para un SVM (Xtrain tiene 3200 datos 3 dimensionales, y_train son las etiquetas). Como podría acelerar el proceso lo máximo posible (quiza con recursos de CPU, RAM, asignando más nucleos o hilos a la ejecución, usando un servidor, etc) dame todas las ideas y posibilidades para poder calcular esto lo mas rapido posible

In [ ]:
#------------------------------------------------
#Creación del kernel cuántico usando parametros entrenados (anteriores)
#------------------------------------------------
def circuito_embedding(params, x, n):
    for p in params:
        #Rotaciones
        for i in range(n):
            qml.Rot(*x, wires=i)
            qml.Rot(*p, wires=i)
        
        #cascada de E (cnots)
        for i in range(n-1): qml.CNOT(wires=[i,i+1]) 

#adjunto
adjunto_empedding=qml.adjoint(circuito_embedding)
n=3
dev3 = qml.device("lightning.qubit", wires=n)
wires = dev3.wires.tolist()

@qml.qnode(dev3)
def circuito_kernel(x1,x2, params,n):
    circuito_embedding(params, x1, n)
    adjunto_empedding(params, x2, n)
    return qml.probs(wires=wires)

def kernel(x1, x2, params,n):
    return circuito_kernel(x1, x2, params,n)[0]

#------------------------------------------------
# Uso del kernel cuántico entrenado en un SVM
#------------------------------------------------

learned_params= params_opt
init_kernel = lambda x1, x2: kernel(x1, x2,learned_params, n)
from sklearn.svm import SVC
svm = SVC(kernel=lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, init_kernel)).fit(X_train,y_train) 

#Accuracy 
def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

nqk_accuracy = accuracy(svm, X_test, y_test)